In [25]:
import matplotlib.pyplot as plt
import librosa
import numpy as np
import pandas as pd
import os
import IPython.display as ipd
%matplotlib inline

In [26]:
root_path="C:/Users/sarib/Desktop/Gohm/SpeechRecognition/database"

In [27]:
def mfcc_extractor(file):
    signal, sample_rate = librosa.load(file)
    ipd.Audio(file)
    mfccs_features = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    
    return mfccs_scaled_features

In [28]:
#NOT USED
def fft_extractor(file):
    signal, sample_rate = librosa.load(file)
    ipd.Audio(file)
    fft = np.fft.fft(signal)
    
    return fft

In [29]:
#NOT USED
signal, sample_rate = librosa.load('database/ac/ac_AHCG_FSRPNCW.wav')
ipd.Audio('database/ac/ac_AHCG_FSRPNCW.wav')
fft = np.fft.fft(signal)
fft

array([ 3.96959355+0.j        , -1.57368256-0.43014124j,
       -0.5562213 +0.51962332j, ...,  0.00995858+0.12657352j,
       -0.5562213 -0.51962332j, -1.57368256+0.43014124j])

In [30]:
#NOT USED
signal, sample_rate = librosa.load('database\sol\sol_AKZI_FAWCVUT.wav' )
ipd.Audio('database\sol\sol_AKZI_FAWCVUT.wav' )
mfccs_features = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40)
mfccs_features.shape

(40, 44)

In [31]:
extracted_feataures =[]
for root, directories, files in os.walk(root_path): 
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names))
            data = mfcc_extractor(os.path.join(root,file_names))
            extracted_feataures.append([data,os.path.basename(os.path.normpath(root))])

In [32]:
extracted_feataures_dataframe= pd.DataFrame(extracted_feataures, columns=["mfcc", "word"])
extracted_feataures_dataframe.head()

,mfcc,word
0,"[-483.0338, 62.193104, -10.424236, 17.89255, -...",ac
1,"[-393.0876, 61.467396, -20.944004, 18.326431, ...",ac
2,"[-557.86304, 99.75818, -6.094904, 0.65864086, ...",ac
3,"[-518.9073, 52.873825, -3.625966, 5.807297, -9...",ac
4,"[-539.5348, 84.21058, -6.7271714, 4.8634925, -...",ac


In [33]:
X=np.array(extracted_feataures_dataframe['mfcc'].tolist())
Y=np.array(extracted_feataures_dataframe['word'].tolist())

In [34]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
Y_numerical=to_categorical(labelencoder.fit_transform(Y))

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(X,Y_numerical, test_size=0.2, random_state=0)

In [36]:
#NOT USED
X_train.shape

(21153, 40)

In [37]:
#NOT USED
len(Y_test)

5289

ANN Model

In [38]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [39]:
model= Sequential()

model.add(Flatten(input_shape=(40, )))
model.add(Dense(300, Activation('relu')))
model.add(Dropout(0.2))
model.add(Dense(300, Activation('relu')))
model.add(Dense(Y_numerical.shape[1], Activation('softmax')))

"""
model.add(Dense(200, input_shape=(40,)))
model.add(Activation('relu'))

model.add(Dense(300))
model.add(Activation('relu'))

model.add(Dense(50))
model.add(Activation('relu'))

model.add(Dense(Y_numerical.shape[1]))
model.add(Activation('sigmoid'))"""

"\nmodel.add(Dense(200, input_shape=(40,)))\nmodel.add(Activation('relu'))\n\nmodel.add(Dense(300))\nmodel.add(Activation('relu'))\n\nmodel.add(Dense(50))\nmodel.add(Activation('relu'))\n\nmodel.add(Dense(Y_numerical.shape[1]))\nmodel.add(Activation('sigmoid'))"

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 40)                0         
                                                                 
 dense_3 (Dense)             (None, 300)               12300     
                                                                 
 dropout_1 (Dropout)         (None, 300)               0         
                                                                 
 dense_4 (Dense)             (None, 300)               90300     
                                                                 
 dense_5 (Dense)             (None, 14)                4214      
                                                                 
Total params: 106,814
Trainable params: 106,814
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=Adam())

In [42]:
checkpointer = ModelCheckpoint(filepath='audio_classification.hdf5', 
                               verbose=0, save_best_only=True)

model.fit(X_train, Y_train, epochs=70, validation_data=(X_test, Y_test), callbacks=[checkpointer], verbose=2)



Epoch 1/70
662/662 - 2s - loss: 3.6760 - accuracy: 0.2521 - val_loss: 1.6457 - val_accuracy: 0.4447 - 2s/epoch - 3ms/step
Epoch 2/70
662/662 - 2s - loss: 1.6829 - accuracy: 0.4231 - val_loss: 1.4605 - val_accuracy: 0.5041 - 2s/epoch - 3ms/step
Epoch 3/70
662/662 - 2s - loss: 1.5226 - accuracy: 0.4725 - val_loss: 1.3449 - val_accuracy: 0.5521 - 2s/epoch - 2ms/step
Epoch 4/70
662/662 - 2s - loss: 1.4013 - accuracy: 0.5141 - val_loss: 1.2155 - val_accuracy: 0.5814 - 2s/epoch - 3ms/step
Epoch 5/70
662/662 - 2s - loss: 1.3102 - accuracy: 0.5444 - val_loss: 1.1626 - val_accuracy: 0.5978 - 2s/epoch - 2ms/step
Epoch 6/70
662/662 - 2s - loss: 1.2465 - accuracy: 0.5641 - val_loss: 1.1102 - val_accuracy: 0.6177 - 2s/epoch - 2ms/step
Epoch 7/70
662/662 - 2s - loss: 1.2002 - accuracy: 0.5830 - val_loss: 1.0763 - val_accuracy: 0.6245 - 2s/epoch - 2ms/step
Epoch 8/70
662/662 - 2s - loss: 1.1417 - accuracy: 0.5983 - val_loss: 1.0600 - val_accuracy: 0.6292 - 2s/epoch - 2ms/step
Epoch 9/70
662/662 - 2s 

In [43]:
print("Test Score: ", model.evaluate(X_test,Y_test))
print("Train Score: ", model.evaluate(X_train,Y_train))

166/166 [==============================] - 0s 1ms/step - loss: 0.8662 - accuracy: 0.7533
Test Score:  [0.8661604523658752, 0.7532615065574646]
662/662 [==============================] - 1s 2ms/step - loss: 0.2128 - accuracy: 0.9302
Train Score:  [0.21279236674308777, 0.9302226901054382]


PREDICTION WITH DATA

In [44]:
predictiondata_feataures =[]
counter_true=0
for root, directories, files in os.walk("C:/Users/sarib/Desktop/Gohm/SpeechRecognition/RecordsFromDataset"): 
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names) )
            r = mfcc_extractor(os.path.join(root,file_names))
            rl=r.reshape(1,-1)
            predicted_label=np.argmax(model.predict(rl), axis=-1)
            prediction_class = labelencoder.inverse_transform(predicted_label) 
            predictiondata_feataures.append([os.path.basename(os.path.normpath(root)), prediction_class, predicted_label])
            if os.path.basename(os.path.normpath(root))==prediction_class:
                counter_true+=1

In [45]:
predictiondata_feataures_dataframe= pd.DataFrame(predictiondata_feataures, columns=["actual data", "predicted data", "predicted data code"])
percentage= (counter_true/len(predictiondata_feataures))*100
print("The percantage of the true predicted data: ", percentage,"%")
#predictiondata_feataures_dataframe.to_csv("output.csv")
predictiondata_feataures_dataframe


The percantage of the true predicted data:  57.14285714285714 %


,actual data,predicted data,predicted data code
0,ac,[ac],[0]
1,ac,[hayir],[7]
2,ac,[sag],[11]
3,asagi,[yukari],[13]
4,asagi,[evet],[5]
5,asagi,[asagi],[1]
6,baslat,[baslat],[2]
7,baslat,[hayir],[7]
8,baslat,[baslat],[2]
9,devam,[devam],[3]


In [46]:
predictiondatas_feataures =[]
counters_true=0
for root, directories, files in os.walk("C:/Users/sarib/Desktop/Gohm/SpeechRecognition/Records"): 
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names) )
            r = mfcc_extractor(os.path.join(root,file_names))
            rl=r.reshape(1,-1)
            predicteds_label=np.argmax(model.predict(rl), axis=-1)
            predictions_class = labelencoder.inverse_transform(predicteds_label) 
            predictiondatas_feataures.append([os.path.basename(os.path.normpath(root)), predictions_class, predicteds_label])
            if os.path.basename(os.path.normpath(root))==predictions_class:
                counters_true+=1

In [47]:
predictiondatas_feataures_dataframe= pd.DataFrame(predictiondatas_feataures, columns=["actual data", "predicted data", "predicted data code"])
percentages= (counters_true/len(predictiondatas_feataures))*100
print("The percantage of the true predicted data: ", percentages,"%")
#predictiondata_feataures_dataframe.to_csv("output.csv")
predictiondatas_feataures_dataframe


The percantage of the true predicted data:  69.04761904761905 %


,actual data,predicted data,predicted data code
0,ac,[ac],[0]
1,ac,[sol],[12]
2,ac,[ac],[0]
3,asagi,[asagi],[1]
4,asagi,[asagi],[1]
5,asagi,[asagi],[1]
6,baslat,[baslat],[2]
7,baslat,[baslat],[2]
8,baslat,[baslat],[2]
9,devam,[devam],[3]


In [48]:
"""filename="predictionData/dursultan.wav"
r=mfcc_extractor(filename)
print(r)
rl=r.reshape(1,-1)
print(rl)
print(rl.shape)
predicted_label=np.argmax(model.predict(rl), axis=-1)

print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
prediction_class"""

'filename="predictionData/dursultan.wav"\nr=mfcc_extractor(filename)\nprint(r)\nrl=r.reshape(1,-1)\nprint(rl)\nprint(rl.shape)\npredicted_label=np.argmax(model.predict(rl), axis=-1)\n\nprint(predicted_label)\nprediction_class = labelencoder.inverse_transform(predicted_label) \nprediction_class'